In [2]:
# 04-train-deep-learning-models.ipynb

# 1. Imports
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.utils.class_weight import compute_class_weight
import pickle
import os

# Create directories if not present
os.makedirs("models", exist_ok=True)

# 2. Load preprocessed datasets
train = pd.read_csv("processed_data/train.csv")
val = pd.read_csv("processed_data/val.csv")
test = pd.read_csv("processed_data/test.csv")

X_train = train.drop("G3_binary", axis=1)
y_train = train["G3_binary"]
X_val = val.drop("G3_binary", axis=1)
y_val = val["G3_binary"]
X_test = test.drop("G3_binary", axis=1)
y_test = test["G3_binary"]

# 3. Compute class weights
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(y_train),
                                     y=y_train)
class_weights_dict = {0: class_weights[0], 1: class_weights[1]}
print("Class weights:", class_weights_dict)

# 4. Load best hyperparameters from tuning results
seq_params = pd.read_csv("results/sequential_hyperparameter_results.csv").sort_values(
    by=["F1-Score", "ROC-AUC"], ascending=False).iloc[0]

wide_deep_params = pd.read_csv("results/widendeep_hyperparameter_results.csv").sort_values(
    by=["F1-Score", "ROC-AUC"], ascending=False).iloc[0]

print("✅ Loaded best Sequential hyperparameters:", seq_params.to_dict())
print("✅ Loaded best Wide & Deep hyperparameters:", wide_deep_params.to_dict())

# ---------------------------------------------------
# 5. Train Final Sequential Model
# ---------------------------------------------------
print("\nTraining Final Sequential Model...")

seq_model = keras.Sequential()
seq_model.add(layers.Input(shape=(X_train.shape[1],)))
for units in eval(seq_params["Layers"]):
    seq_model.add(layers.Dense(int(units), activation='relu'))
    if float(seq_params["Dropout"]) > 0:
        seq_model.add(layers.Dropout(float(seq_params["Dropout"])))
seq_model.add(layers.Dense(1, activation='sigmoid'))

optimizer = keras.optimizers.Adam(learning_rate=float(seq_params["Learning Rate"]))
seq_model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=["accuracy", keras.metrics.AUC(name="auc")])

seq_model.fit(X_train, y_train,
              validation_data=(X_val, y_val),
              epochs=50,
              batch_size=32,
              class_weight=class_weights_dict,
              verbose=1)

seq_model.save("models/sequential_model.h5")
print("✅ Sequential model saved to models/sequential_model.h5")

# ---------------------------------------------------
# 6. Train Final Wide & Deep Model
# ---------------------------------------------------
print("\nTraining Final Wide & Deep Model...")

# Identify categorical and numeric columns
cat_cols = [col for col in X_train.columns if any(prefix in col for prefix in 
    ['school_', 'sex_', 'address_', 'famsize_', 'Pstatus_', 'Mjob_', 'Fjob_', 
     'reason_', 'guardian_', 'schoolsup_', 'famsup_', 'paid_', 'activities_', 
     'nursery_', 'higher_', 'internet_', 'romantic_'])]
num_cols = [col for col in X_train.columns if col not in cat_cols]

X_train_wide, X_val_wide, X_test_wide = X_train[cat_cols], X_val[cat_cols], X_test[cat_cols]
X_train_deep, X_val_deep, X_test_deep = X_train[num_cols], X_val[num_cols], X_test[num_cols]

# Build Wide & Deep
input_wide = layers.Input(shape=(X_train_wide.shape[1],), name="wide_input")
input_deep = layers.Input(shape=(X_train_deep.shape[1],), name="deep_input")

deep = input_deep
for units in eval(str(wide_deep_params["Layers"])):
    deep = layers.Dense(int(units), activation='relu')(deep)
    if float(wide_deep_params["Dropout"]) > 0:
        deep = layers.Dropout(float(wide_deep_params["Dropout"]))(deep)

combined = layers.concatenate([input_wide, deep])
output = layers.Dense(1, activation='sigmoid')(combined)

wide_deep_model = keras.Model(inputs=[input_wide, input_deep], outputs=output)

optimizer = keras.optimizers.Adam(learning_rate=float(wide_deep_params["Learning Rate"]))
wide_deep_model.compile(optimizer=optimizer,
                        loss='binary_crossentropy',
                        metrics=['accuracy', keras.metrics.AUC(name='auc')])

wide_deep_model.fit([X_train_wide, X_train_deep], y_train,
                    validation_data=([X_val_wide, X_val_deep], y_val),
                    epochs=50, batch_size=32,
                    class_weight=class_weights_dict,
                    verbose=1)

wide_deep_model.save("models/wide_deep_model.h5")
print("✅ Wide & Deep model saved to models/wide_deep_model.h5")


Class weights: {0: np.float64(0.6133333333333333), 1: np.float64(2.7058823529411766)}
✅ Loaded best Sequential hyperparameters: {'Model No.': 5, 'Layers': '[16, 8]', 'Learning Rate': 0.001, 'Dropout': 0.0, 'Accuracy': 0.7333333333333333, 'Precision': 0.3809523809523809, 'Recall': 0.7272727272727273, 'F1-Score': 0.5, 'ROC-AUC': 0.7847866419294991}
✅ Loaded best Wide & Deep hyperparameters: {'Model No.': 3, 'Layers': '[10]', 'Learning Rate': 0.01, 'Dropout': 0.0, 'Accuracy': 0.7666666666666667, 'Precision': 0.3846153846153846, 'Recall': 0.4545454545454545, 'F1-Score': 0.4166666666666667, 'ROC-AUC': 0.7105751391465678}

Training Final Sequential Model...
Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.6495 - auc: 0.3732 - loss: 0.7560 - val_accuracy: 0.5424 - val_auc: 0.4044 - val_loss: 0.6629
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.5693 - auc: 0.4039 - loss: 0.6872 - val_accuracy: 0.5085 - val_auc: 0.4081 - val_loss: 0.6724
Epoch 3/50
9/9 ━━━━━━━━━━━

✅ Sequential model saved to models/sequential_model.h5

Training Final Wide & Deep Model...
Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.2713 - auc: 0.4630 - loss: 0.8620 - val_accuracy: 0.5593 - val_auc: 0.6534 - val_loss: 0.7142
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.5384 - auc: 0.5082 - loss: 0.7287 - val_accuracy: 0.6441 - val_auc: 0.7614 - val_loss: 0.5939
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.6660 - auc: 0.6710 - loss: 0.6404 - val_accuracy: 0.6610 - val_auc: 0.7756 - val_loss: 0.6202
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6337 - auc: 0.7319 - loss: 0.6531 - val_accuracy: 0.6102 - val_auc: 0.7822 - val_loss: 0.6925
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.5262 - auc: 0.7200 - loss: 0.6314 - val_accuracy: 0.6610 - val_auc: 0.7898 - val_loss: 0.5909
Epoch 6/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6844 - auc: 0.7465 - loss: 0.5880 - val_accuracy: 0.7288 - val_

✅ Wide & Deep model saved to models/wide_deep_model.h5
